In [1]:
import gzip
import json
import time
import requests 
import pandas as pd
import logging as log
from configparser import RawConfigParser

class Collector():
    
    URL = 'https://neuvoo.com/services/api-new/search?' + \
        'ip=1.1.1.1&useragent=123asd&' + \
        'k=${JOB_TITLE}& ' + \
        'l=${CITY}&f=&o=&p=&r=15&' + \
        'country=${COUNTRY}' + \
        '&format=json&jobdesc=1' + \
        '&start=${START}' + \
        '&limit=${LIMIT}'

    #limit jobs post per resquest (defalt and max = 15)
    LIMIT = 15
    
    #max jobs post per occupation
    MAX = 1000
       
    def __init__(self, args):
        #credentials file
        config = RawConfigParser()
        config.read('./credentials.properties')
     
        self.args = args
        self.username = config.get('ApiSection', 'neuvoo.username')
        self.password = config.get('ApiSection', 'neuvoo.password')
  
        #print(self.username, self.password, self.args['input_file'], self.args['output_file'], self.URL)
    
    def get_job_request(self, job_title, start, limit, location):
        #country, city = location.split(',')
        return requests.get(self.URL
                            .replace("${JOB_TITLE}", job_title)
                            .replace("${START}", start)
                            .replace("${LIMIT}", limit)
                            .replace("${COUNTRY}", 'FR')
                            .replace("${CITY}", location),
                            auth=(self.username, self.password))
    

    def get_jobs(self, job_title, location):
        print("Getting jobs for: " + job_title + " - "+ location)
        data = pd.DataFrame()
        job_titles = [] 
        job_titles_found = []
        companies = []
        descriptions = []
        locations = []
        sources = [] 
        jobkeys = []
        dates = []
        department_name = []
        department_code = []

        for i in range(0, self.MAX, self.LIMIT):
            results = self.get_job_request(job_title, str(i), str(self.LIMIT), location)
            #results_count = len(results.json()['results'])
            totalresults = results.json()['totalresults']
            if totalresults != None:
                for result in results.json()['results']:
                    job_titles.append(job_title)
                    job_titles_found.append(result['jobtitle'])
                    companies.append(result['company'])
                    descriptions.append(result['description'])
                    locations.append(result['formattedLocation'])
                    sources.append(result['source'])
                    jobkeys.append(result['jobkey'])
                    dates.append(result['date'])
                    department_name.append(location)
                    department_code.append(get_department_code(location))
            else:
                break

        data["job_title"] = job_titles
        data["job_title_found"] = job_titles_found
        data["company"] = companies
        data["location"] = locations
        data["sources"] = sources
        data["jobkeys"] = jobkeys
        data["dates"] = dates
        data["description"] = descriptions
        data["department_name"] = department_name
        data["department_code"] = department_code

        return data
    
    def run(self):
        ts = time.gmtime()
        df = pd.DataFrame()
        for location in self.args['locations']:
            df = df.append([self.get_jobs(job_title, location) for job_title in self.args['jobs']])
        
        #tmp_output_file = args['output_file'].replace("${TS}", time.strftime("%Y%m%d%H%M%S", ts))
        #df.to_csv(tmp_output_file, index=False, compression='gzip')

        #print("File saved: ", tmp_output_file)

        return df

In [2]:
departments = pd.read_excel('data/department code.xlsx')
departments = departments[1:]
departments

,NUMÉRO,NOM
1,1,Ain
2,2,Aisne
3,3,Allier
4,4,Alpes-de-Haute-Provence
5,5,Hautes-Alpes
6,6,Alpes-Maritimes
7,7,Ardèche
8,8,Ardennes
9,9,Ariège
10,10,Aube


In [3]:
def get_department_code(name):
    return departments[departments['NOM'].str.startswith(name)]['NUMÉRO'].values[0]
    
get_department_code('Somme')

80

In [4]:
rome_codes = pd.read_csv('data/Sncf codes romes - Sheet1.csv', names=['title', 'rome_code'])
rome_codes.head()

,title,rome_code
0,Agent technique d'unité patrimoine,C1501
1,Assistant technique Patrimoine,C1501
2,Gestionnaire technique immobilier,C1501
3,Assistant de production,C1502
4,Gestionnaire du fichier immobilier,C1502


In [5]:
def get_rome_code(title):
    return rome_codes[rome_codes['title'].str.startswith(title)]['rome_code'].values[0]
    
get_rome_code('Assistant de production')

'C1502'

In [9]:
JOB_TITLES = rome_codes['title'][80:95]
#JOB_TITLES = ['Assistant de production']

LOCATIONS = departments['NOM'].values
#LOCATIONS = ['Paris']

#arguments
args = {
    'name':'neuvoo',
    'jobs': JOB_TITLES,
    'locations': LOCATIONS,
    'output_file': '../../../data/raw/neuvoo/${TS}.csv.gz',
    #...
}

collector = Collector(args)
df = collector.run()
df

Getting jobs for: Responsable de pôle des Services Généraux - Ain
Getting jobs for: Dirigeant comptable - Ain
Getting jobs for: Responsable équipe comptable - Ain
Getting jobs for: Consultant interne en management et accompagnement du changement - Ain
Getting jobs for: Chargé d'études marketing marchés et développement - Ain
Getting jobs for: Chargé veille stratégique et prospective Voy - Ain
Getting jobs for: Chargé études RH - Ain
Getting jobs for: Conseiller carrière - Ain
Getting jobs for: Conseiller Mobilité - Ain
Getting jobs for: Coordinateur Formation - Ain
Getting jobs for: Gestionnaire de documentation Services Généraux - Ain
Getting jobs for: Gestionnaire de l'emploi - Ain
Getting jobs for: Resp Centre Pilotage Formation - Ain
Getting jobs for: Responsable de pôle gestion - Ain
Getting jobs for: Responsable Développement des Compétences et Formation - Ain
Getting jobs for: Responsable de pôle des Services Généraux - Aisne
Getting jobs for: Dirigeant comptable - Aisne
Getting

Getting jobs for: Chargé veille stratégique et prospective Voy - Ariège
Getting jobs for: Chargé études RH - Ariège
Getting jobs for: Conseiller carrière - Ariège
Getting jobs for: Conseiller Mobilité - Ariège
Getting jobs for: Coordinateur Formation - Ariège
Getting jobs for: Gestionnaire de documentation Services Généraux - Ariège
Getting jobs for: Gestionnaire de l'emploi - Ariège
Getting jobs for: Resp Centre Pilotage Formation - Ariège
Getting jobs for: Responsable de pôle gestion - Ariège
Getting jobs for: Responsable Développement des Compétences et Formation - Ariège
Getting jobs for: Responsable de pôle des Services Généraux - Aube
Getting jobs for: Dirigeant comptable - Aube
Getting jobs for: Responsable équipe comptable - Aube
Getting jobs for: Consultant interne en management et accompagnement du changement - Aube
Getting jobs for: Chargé d'études marketing marchés et développement - Aube
Getting jobs for: Chargé veille stratégique et prospective Voy - Aube
Getting jobs for

Getting jobs for: Responsable Développement des Compétences et Formation - Charente-Maritime
Getting jobs for: Responsable de pôle des Services Généraux - Cher
Getting jobs for: Dirigeant comptable - Cher
Getting jobs for: Responsable équipe comptable - Cher
Getting jobs for: Consultant interne en management et accompagnement du changement - Cher
Getting jobs for: Chargé d'études marketing marchés et développement - Cher
Getting jobs for: Chargé veille stratégique et prospective Voy - Cher
Getting jobs for: Chargé études RH - Cher
Getting jobs for: Conseiller carrière - Cher
Getting jobs for: Conseiller Mobilité - Cher
Getting jobs for: Coordinateur Formation - Cher
Getting jobs for: Gestionnaire de documentation Services Généraux - Cher
Getting jobs for: Gestionnaire de l'emploi - Cher
Getting jobs for: Resp Centre Pilotage Formation - Cher
Getting jobs for: Responsable de pôle gestion - Cher
Getting jobs for: Responsable Développement des Compétences et Formation - Cher
Getting jobs 

Getting jobs for: Chargé études RH - Doubs
Getting jobs for: Conseiller carrière - Doubs
Getting jobs for: Conseiller Mobilité - Doubs
Getting jobs for: Coordinateur Formation - Doubs
Getting jobs for: Gestionnaire de documentation Services Généraux - Doubs
Getting jobs for: Gestionnaire de l'emploi - Doubs
Getting jobs for: Resp Centre Pilotage Formation - Doubs
Getting jobs for: Responsable de pôle gestion - Doubs
Getting jobs for: Responsable Développement des Compétences et Formation - Doubs
Getting jobs for: Responsable de pôle des Services Généraux - Drôme
Getting jobs for: Dirigeant comptable - Drôme
Getting jobs for: Responsable équipe comptable - Drôme
Getting jobs for: Consultant interne en management et accompagnement du changement - Drôme
Getting jobs for: Chargé d'études marketing marchés et développement - Drôme
Getting jobs for: Chargé veille stratégique et prospective Voy - Drôme
Getting jobs for: Chargé études RH - Drôme
Getting jobs for: Conseiller carrière - Drôme
Ge

Getting jobs for: Consultant interne en management et accompagnement du changement - Hérault
Getting jobs for: Chargé d'études marketing marchés et développement - Hérault
Getting jobs for: Chargé veille stratégique et prospective Voy - Hérault
Getting jobs for: Chargé études RH - Hérault
Getting jobs for: Conseiller carrière - Hérault
Getting jobs for: Conseiller Mobilité - Hérault
Getting jobs for: Coordinateur Formation - Hérault
Getting jobs for: Gestionnaire de documentation Services Généraux - Hérault
Getting jobs for: Gestionnaire de l'emploi - Hérault
Getting jobs for: Resp Centre Pilotage Formation - Hérault
Getting jobs for: Responsable de pôle gestion - Hérault
Getting jobs for: Responsable Développement des Compétences et Formation - Hérault
Getting jobs for: Responsable de pôle des Services Généraux - Ille-et-Vilaine
Getting jobs for: Dirigeant comptable - Ille-et-Vilaine
Getting jobs for: Responsable équipe comptable - Ille-et-Vilaine
Getting jobs for: Consultant interne 

...

Getting jobs for: Dirigeant comptable - Haut-Rhin
Getting jobs for: Responsable équipe comptable - Haut-Rhin
Getting jobs for: Consultant interne en management et accompagnement du changement - Haut-Rhin
Getting jobs for: Chargé d'études marketing marchés et développement - Haut-Rhin
Getting jobs for: Chargé veille stratégique et prospective Voy - Haut-Rhin
Getting jobs for: Chargé études RH - Haut-Rhin
Getting jobs for: Conseiller carrière - Haut-Rhin
Getting jobs for: Conseiller Mobilité - Haut-Rhin
Getting jobs for: Coordinateur Formation - Haut-Rhin
Getting jobs for: Gestionnaire de documentation Services Généraux - Haut-Rhin
Getting jobs for: Gestionnaire de l'emploi - Haut-Rhin
Getting jobs for: Resp Centre Pilotage Formation - Haut-Rhin
Getting jobs for: Responsable de pôle gestion - Haut-Rhin
Getting jobs for: Responsable Développement des Compétences et Formation - Haut-Rhin
Getting jobs for: Responsable de pôle des Services Généraux - Rhône
Getting jobs for: Dirigeant comptab

Getting jobs for: Coordinateur Formation - Seine-Maritime
Getting jobs for: Gestionnaire de documentation Services Généraux - Seine-Maritime
Getting jobs for: Gestionnaire de l'emploi - Seine-Maritime
Getting jobs for: Resp Centre Pilotage Formation - Seine-Maritime
Getting jobs for: Responsable de pôle gestion - Seine-Maritime
Getting jobs for: Responsable Développement des Compétences et Formation - Seine-Maritime
Getting jobs for: Responsable de pôle des Services Généraux - Seine-et-Marne
Getting jobs for: Dirigeant comptable - Seine-et-Marne
Getting jobs for: Responsable équipe comptable - Seine-et-Marne
Getting jobs for: Consultant interne en management et accompagnement du changement - Seine-et-Marne
Getting jobs for: Chargé d'études marketing marchés et développement - Seine-et-Marne
Getting jobs for: Chargé veille stratégique et prospective Voy - Seine-et-Marne
Getting jobs for: Chargé études RH - Seine-et-Marne
Getting jobs for: Conseiller carrière - Seine-et-Marne
Getting job

Getting jobs for: Responsable équipe comptable - Vendée
Getting jobs for: Consultant interne en management et accompagnement du changement - Vendée
Getting jobs for: Chargé d'études marketing marchés et développement - Vendée
Getting jobs for: Chargé veille stratégique et prospective Voy - Vendée
Getting jobs for: Chargé études RH - Vendée
Getting jobs for: Conseiller carrière - Vendée
Getting jobs for: Conseiller Mobilité - Vendée
Getting jobs for: Coordinateur Formation - Vendée
Getting jobs for: Gestionnaire de documentation Services Généraux - Vendée
Getting jobs for: Gestionnaire de l'emploi - Vendée
Getting jobs for: Resp Centre Pilotage Formation - Vendée
Getting jobs for: Responsable de pôle gestion - Vendée
Getting jobs for: Responsable Développement des Compétences et Formation - Vendée
Getting jobs for: Responsable de pôle des Services Généraux - Vienne
Getting jobs for: Dirigeant comptable - Vienne
Getting jobs for: Responsable équipe comptable - Vienne
Getting jobs for: Co

Getting jobs for: Conseiller carrière - Seine-St-Denis
Getting jobs for: Conseiller Mobilité - Seine-St-Denis
Getting jobs for: Coordinateur Formation - Seine-St-Denis
Getting jobs for: Gestionnaire de documentation Services Généraux - Seine-St-Denis
Getting jobs for: Gestionnaire de l'emploi - Seine-St-Denis
Getting jobs for: Resp Centre Pilotage Formation - Seine-St-Denis
Getting jobs for: Responsable de pôle gestion - Seine-St-Denis
Getting jobs for: Responsable Développement des Compétences et Formation - Seine-St-Denis
Getting jobs for: Responsable de pôle des Services Généraux - Val-de-Marne
Getting jobs for: Dirigeant comptable - Val-de-Marne
Getting jobs for: Responsable équipe comptable - Val-de-Marne
Getting jobs for: Consultant interne en management et accompagnement du changement - Val-de-Marne
Getting jobs for: Chargé d'études marketing marchés et développement - Val-de-Marne
Getting jobs for: Chargé veille stratégique et prospective Voy - Val-de-Marne
Getting jobs for: Ch

,job_title,job_title_found,company,location,sources,jobkeys,dates,description,department_name,department_code
0,Responsable équipe comptable,Responsable d’équipe comptable,Ordre des Experts-Comptables,"Bellegarde-sur-Valserine, Auvergne-Rhône-Alpes...",Ordre des Experts-Comptables,f6a1b9064872,2020-07-15,Entreprise :CERFRANCE Ain Description du poste...,Ain,1.0
1,Responsable équipe comptable,RESPONSABLE D'ÉQUIPE COMPTABLE,Cerfrance,"null, Auvergne-Rhône-Alpes, FR",cerfrance,1906d30e19ea,2020-07-15,Missions Le candidat sera le référent comptabl...,Ain,1.0
0,Gestionnaire de l'emploi,GESTIONNAIRE APPUI Agence de Nice Nord F/H,Pôle emploi,"Nice, Provence-Alpes-Côte d'Azur, FR",Pôle emploi,15f91fd8252f,2020-07-13,Temps de travailTemps completMissions Dans le ...,Alpes-Maritimes,6.0
0,Gestionnaire de l'emploi,GESTIONNAIRE DE PARC DE CONTENEURS H/F ...,orano,"Vendeuvre-sur-Barse, Grand Est, FR",Energie recrute,fbfc9ae9f497,2020-07-16,Description du poste Intitulé du poste ...,Aube,10.0
1,Gestionnaire de l'emploi,GESTIONNAIRE DE SITE INDUSTRIEL H/F ...,EDF,"Nogent-sur-Seine, Grand Est, FR",Energie recrute,157d5aec3e5d,2020-07-16,Description du poste Type de contrat : C...,Aube,10.0
0,Conseiller Mobilité,CONSEILLER DE VENTE CYCLE / MOBILITE URBAINE H/F,INTERSPORT,"Salon-de-Provence, Provence-Alpes-Côte d'Azur, FR",Clic & Sport,437687adeb6d,2020-07-14,Nous recherchons un(e) CONSEILLER DE VENTE CYC...,Bouches-du-Rhône,13.0
0,Gestionnaire de l'emploi,Gestionnaire de bases de donn&eacute;es et de ...,Pôle emploi,"Martigues, Provence-Alpes-Côte d'Azur, FR",Clic & Power,6a791723fa51,2020-07-16,Nous recherchons un(e) Gestionnaire de bases d...,Bouches-du-Rhône,13.0
0,Responsable équipe comptable,RESPONSABLE D'ÉQUIPE COMPTABLE,Cerfrance,"null, Lower Normandy, FR",cerfrance,974e11034025,2020-07-15,"Missions Intégré à une équipe comptable, vous ...",Calvados,14.0
0,Chargé études RH,Charge d'etudes statistiques rh,Fédération Hospitalière de France,"La Rochelle, Nouvelle-Aquitaine, FR",Fédération Hospitalière de France,e5d5cbdac739,2020-07-13,Descriptif Le Groupe Hospitalier de La Rochell...,Charente,16.0
0,Gestionnaire de l'emploi,Emploi gestionnaire de paie angoulême h/f,Expectra,"Angoulême, Nouvelle-Aquitaine, FR",Keljob,0b864e5d2a16,2020-07-12,"<h2>Le poste</h2>Vos missions : évidemment, vo...",Charente,16.0


In [10]:
df['rome_code'] = [get_rome_code(title) for title in df['job_title']]
df.head()

,job_title,job_title_found,company,location,sources,jobkeys,dates,description,department_name,department_code,rome_code
0,Responsable équipe comptable,Responsable d’équipe comptable,Ordre des Experts-Comptables,"Bellegarde-sur-Valserine, Auvergne-Rhône-Alpes...",Ordre des Experts-Comptables,f6a1b9064872,2020-07-15,Entreprise :CERFRANCE Ain Description du poste...,Ain,1.0,M1206
1,Responsable équipe comptable,RESPONSABLE D'ÉQUIPE COMPTABLE,Cerfrance,"null, Auvergne-Rhône-Alpes, FR",cerfrance,1906d30e19ea,2020-07-15,Missions Le candidat sera le référent comptabl...,Ain,1.0,M1206
0,Gestionnaire de l'emploi,GESTIONNAIRE APPUI Agence de Nice Nord F/H,Pôle emploi,"Nice, Provence-Alpes-Côte d'Azur, FR",Pôle emploi,15f91fd8252f,2020-07-13,Temps de travailTemps completMissions Dans le ...,Alpes-Maritimes,6.0,M1502
0,Gestionnaire de l'emploi,GESTIONNAIRE DE PARC DE CONTENEURS H/F ...,orano,"Vendeuvre-sur-Barse, Grand Est, FR",Energie recrute,fbfc9ae9f497,2020-07-16,Description du poste Intitulé du poste ...,Aube,10.0,M1502
1,Gestionnaire de l'emploi,GESTIONNAIRE DE SITE INDUSTRIEL H/F ...,EDF,"Nogent-sur-Seine, Grand Est, FR",Energie recrute,157d5aec3e5d,2020-07-16,Description du poste Type de contrat : C...,Aube,10.0,M1502


In [11]:
df.to_clipboard(index=False, header=False)

In [19]:
df.loc[df.job_title == 'promoteur des ventes', 'rome_code'] = 'M1706'
df.loc[df.job_title == 'responsable de secteur', 'rome_code'] = 'D1509'

df

,job_title,job_title_found,company,location,sources,jobkeys,dates,description,department_name,department_code,rome_code
0,Télé conseiller (H/F),Conseiller marketing relationnel et e-commerce...,Distributeur Indépendant,"Gex, Auvergne-Rhône-Alpes, FR",PMEjob.fr,4cf804add098,2020-07-14,La société :Nous travaillons en partenariat av...,Ain,1,NaN
0,Télé conseiller (H/F),CONSEILLER DE VENTE CDD - SAINT QUENTIN FAYET H/F,Bouygues Telecom,"Saint-Quentin, Hauts-de-France, FR",Direct Emploi,cfd3ab58c2e9,2020-07-10,#nosclientdabord est devenu un moteur de l&#39...,Aisne,2,NaN
1,Télé conseiller (H/F),Conseiller marketing relationnel et e-commerce...,Distributeur Indépendant,"Laon, Hauts-de-France, FR",Jobposting,b1cf082e6045,2020-07-09,Nous travaillons en partenariat avec une grand...,Aisne,2,NaN
2,Télé conseiller (H/F),Conseiller commercial beauté et e-boutique tél...,Distributeur Indépendant,"Laon, Hauts-de-France, FR",PMEjob.fr,fa237daa7c98,2020-07-11,La société :Nous travaillons en partenariat av...,Aisne,2,NaN
3,Télé conseiller (H/F),Conseillère beauté et télétravail e-boutique H/F,Distributeur Indépendant,"Laon, Hauts-de-France, FR",Jobposting,0464598b79cd,2020-07-12,Nous travaillons en partenariat avec une grand...,Aisne,2,NaN
0,Télé conseiller (H/F),Conseillère beauté e-commerce télétravail H/F,Distributeur Indépendant,"Vichy, Auvergne-Rhône-Alpes, FR",Jobposting,4bd75bd7b799,2020-07-15,Nous travaillons en partenariat avec une grand...,Allier,3,NaN
1,Télé conseiller (H/F),Conseiller commercial beauté et e-boutique tél...,Distributeur Indépendant,"Montluçon, Auvergne-Rhône-Alpes, FR",Jobposting,27c26b48ab02,2020-07-10,Nous travaillons en partenariat avec une grand...,Allier,3,NaN
2,Télé conseiller (H/F),Conseiller marketing relationnel et e-commerce...,Distributeur Indépendant,"Moulins, Auvergne-Rhône-Alpes, FR",Jobposting,d4eec3500987,2020-07-09,Nous travaillons en partenariat avec une grand...,Allier,3,NaN
3,Télé conseiller (H/F),CONSEILLER DE VENTE CDD ETE - MONTLUCON H/F,Bouygues Telecom,"Montluçon, Auvergne-Rhône-Alpes, FR",Bouygues Telecom,24e327c18bac,2020-07-10,Description de la mission Chez Bouygues Teleco...,Allier,3,NaN
0,Télé conseiller (H/F),Conseillère commerciale - Conseiller commercia...,Orange,"Arrondissement of Gap, Provence-Alpes-Côte d'A...",Orange,95d8962fe9b3,2020-07-09,about the role Nous vous proposons un contrat ...,Hautes-Alpes,5,NaN


In [44]:
#Status Code, Content Type, Enconding, Total Results
#r.status_code, r.headers['content-type'], r.encoding, r.json()['totalresults']

result = collector.get_job_request('Data scientist', '0', '15', "us,Silicon Valley")
assert result.status_code == 200
result.json()['results']

In [46]:
collector.get_job_request('ventes', '0', '15', "FR, Paris").json()

{'totalresults': 211,
 'results': [{'jobkey': 'ce45ab23fa3b',
   'jobtitle': 'Chef de ventes Véhicules Neufs H/F Région parisienne',
   'company': 'RENAULT RETAIL GROUP',
   'city': 'Paris',
   'state': 'Île-de-France',
   'country': 'FR',
   'formattedLocation': 'Paris, Île-de-France, FR',
   'source': 'RENAULT RETAIL GROUP',
   'date': '2020-07-02',
   'description': "Des métiers « passion », des produits « plaisir », des possibilités de s'épanouir professionnellement dans un environnement varié et dynamique centré sur nos clients et leur satisfaction…Renault Retail Group est une filiale à 100% du constructeur Renault. Le monde de l'automobile est en perpétuelle évolution et nous nous attachons à la conception de véhicules innovants, séduisants et encore plus respectueux de l'environnement. RRG repose sur nos valeurs telles que : garder toujours le client à l'esprit, apporter sa contribution, parler vrai, apprendre au quotidien et faire simple. Notre culture d'entreprise est notre fo

In [47]:
collector.get_job_request('promotion des ventes', '0', '15', "FR, Paris").json()

{'totalresults': None, 'results': None}